<a href="https://colab.research.google.com/github/amine406/SeoProject/blob/main/Crawling_Multi_thread%C3%A9_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Initialiser les variables
visited_urls = set()  # Pour stocker les URLs visitées
url_info_list = []  # Stocker les infos SEO pour chaque URL

# Créer une session globale
session = requests.Session()

# Ajouter un User-Agent à la session
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
})

# Fonction pour vérifier si l'URL est valide et interne
def is_valid_url(url, base_url):
    parsed_url = urlparse(url)
    return bool(parsed_url.scheme) and bool(parsed_url.netloc) and base_url in url

# Fonction pour crawler et analyser une page
def crawl_page(url, base_url):
    try:
        response = session.get(url, timeout=5)  # Timeout après 5 secondes
        status_code = response.status_code
        print(f"Crawling {url} {status_code}")

        if status_code != 200:
            return []  # Ignorer si le status n'est pas OK

        # Vérifier que la réponse est bien du HTML
        content_type = response.headers.get("Content-Type", "")
        print(f"Content-Type de {url}: {content_type}")
        if "text/html" not in content_type:
            return []  # Ignorer le contenu non-HTML

        # Utiliser un parser robuste
        soup = BeautifulSoup(response.content, "html.parser")

        # Récupérer les informations SEO
        title = soup.title.string if soup.title else "No title"
        description = soup.find("meta", attrs={"name": "description"})
        description = description["content"] if description else "No description"
        h1 = soup.find("h1")
        h1_text = h1.get_text() if h1 else "No H1"

        # Compter le nombre de liens sur la page
        links = soup.find_all("a", href=True)
        num_links = len(links)
        print(f"{num_links} liens trouvés sur {url}")

        # Stocker les informations dans une liste
        url_info_list.append({
            "URL": url,
            "HTTP Status": status_code,
            "Title": title,
            "Meta Description": description,
            "H1": h1_text,
            "Number of Links": num_links
        })

        # Retourner la liste des liens internes
        return [urljoin(base_url, link.get('href')) for link in links if is_valid_url(urljoin(base_url, link.get('href')), base_url)]

    except requests.RequestException as e:
        print(f"Failed to crawl {url}: {e}")
        return []  # Return an empty list if there is an error

# Fonction principale pour démarrer le crawling
def crawl_website(start_url, max_workers=5):
    base_url = "{0.scheme}://{0.netloc}".format(urlparse(start_url))
    to_crawl = [start_url]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []

        while to_crawl:
            url = to_crawl.pop()

            if url not in visited_urls:
                visited_urls.add(url)
                futures.append(executor.submit(crawl_page, url, base_url))

        # Attendre que toutes les tâches soient terminées

        for future in futures:
         new_links = future.result()

       # Print each new link on a separate line
        print("Nouveaux liens trouvés :")
        for link in new_links:
               print(link)

    # Extend the to_crawl list with the new links
    to_crawl.extend(new_links)

    # Convertir les données collectées en DataFrame
    df = pd.DataFrame(url_info_list)

    # Afficher le tableau
    print("\nTableau récapitulatif des pages crawlées:")
    print(df)

    # Exporter vers un fichier CSV si des données ont été collectées
    if not df.empty:
        df.to_csv("seo_crawl_results_file.csv", index=False)
        print("Résultats exportés vers 'seo_crawl_results_file'.")
    else:
        print("Aucune donnée à exporter.")

# Démarrer le crawler sur le site "https://aundetailpres.fr"
if __name__ == "__main__":
    print("Démarrage du crawling sur https://aundetailpres.fr...")
    crawl_website("https://aundetailpres.fr", max_workers=10)
    print("Crawling terminé.")